In [10]:
import tensorflow as tf
import pandas as pd
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split


In [2]:
# import dataset

data = pd.read_csv('Corona_NLP_train.csv', encoding='latin-1')
data

,UserName,ScreenName,Location,TweetAt,OriginalTweet,Sentiment
0,3799,48751,London,16-03-2020,@MeNyrbie @Phil_Gahan @Chrisitv https://t.co/i...,Neutral
1,3800,48752,UK,16-03-2020,advice Talk to your neighbours family to excha...,Positive
2,3801,48753,Vagabonds,16-03-2020,Coronavirus Australia: Woolworths to give elde...,Positive
3,3802,48754,NaN,16-03-2020,My food stock is not the only one which is emp...,Positive
4,3803,48755,NaN,16-03-2020,"Me, ready to go at supermarket during the #COV...",Extremely Negative
...,...,...,...,...,...,...
41152,44951,89903,"Wellington City, New Zealand",14-04-2020,Airline pilots offering to stock supermarket s...,Neutral
41153,44952,89904,NaN,14-04-2020,Response to complaint not provided citing COVI...,Extremely Negative
41154,44953,89905,NaN,14-04-2020,You know itÂs getting tough when @KameronWild...,Positive
41155,44954,89906,NaN,14-04-2020,Is it wrong that the smell of hand sanitizer i...,Neutral


In [3]:
data.drop(columns=['UserName', 'ScreenName', 'Location', 'TweetAt'], inplace=True)

In [4]:
data_train, data_test, label_train, label_test = train_test_split(data.OriginalTweet, data.Sentiment, test_size=0.2, random_state=21 )

In [5]:
len(data_test)

8232

In [6]:
from sklearn.preprocessing import LabelEncoder

encd = LabelEncoder()

encd.fit(label_train)
label_latih = encd.transform(label_train)

encd.fit(label_test)
label_test = encd.transform(label_test)

In [7]:
# tokenizing

tokenizer = Tokenizer(num_words=10000, oov_token='xx')
tokenizer.fit_on_texts(data_train)
tokenizer.fit_on_texts(data_test)
 
sekuens_latih = tokenizer.texts_to_sequences(data_train)
sekuens_test = tokenizer.texts_to_sequences(data_test)
 
padded_latih = pad_sequences(sekuens_latih) 
padded_test = pad_sequences(sekuens_test)

In [8]:
padded_test.shape

(8232, 65)

In [11]:
model_lstm = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=10000, output_dim=32),
    tf.keras.layers.LSTM(64, return_sequences=True),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.LSTM(32),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(5, activation='Softmax'),
])

model_lstm.compile(loss='sparse_categorical_crossentropy',
                    optimizer= tf.optimizers.Adam(0.001),
                    metrics=['accuracy'])


In [12]:
history = model_lstm.fit(padded_latih,
                        label_latih,
                        epochs=20,
                        validation_data=(padded_test, label_test),
                        verbose=1)

Epoch 1/20
1029/1029 [==============================] - 63s 47ms/step - loss: 1.2830 - accuracy: 0.4482 - val_loss: 0.9589 - val_accuracy: 0.6284
Epoch 2/20
1029/1029 [==============================] - 47s 45ms/step - loss: 0.7262 - accuracy: 0.7352 - val_loss: 0.7361 - val_accuracy: 0.7330
Epoch 3/20
1029/1029 [==============================] - 49s 48ms/step - loss: 0.5491 - accuracy: 0.8107 - val_loss: 0.6531 - val_accuracy: 0.7694
Epoch 4/20
1029/1029 [==============================] - 49s 47ms/step - loss: 0.4461 - accuracy: 0.8459 - val_loss: 0.6637 - val_accuracy: 0.7690
Epoch 5/20
1029/1029 [==============================] - 49s 48ms/step - loss: 0.3654 - accuracy: 0.8744 - val_loss: 0.6561 - val_accuracy: 0.7742
Epoch 6/20
1029/1029 [==============================] - 49s 48ms/step - loss: 0.3073 - accuracy: 0.8943 - val_loss: 0.6915 - val_accuracy: 0.7829
Epoch 7/20
1029/1029 [==============================] - 47s 46ms/step - loss: 0.2615 - accuracy: 0.9111 - val_loss: 0.7593 -